In [3]:
import os
import subprocess
import tempfile
from pathlib import Path
import io

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert import HTMLExporter
from nbparameterise import extract_parameters, parameter_values, replace_definitions
from nbconvert.filters import ansi2html, strip_ansi


def _notebook_run(path, SCOPETYPE='OPENADC', PLATFORM='CWLITEARM', **kwargs):
    """Execute a notebook via nbconvert and collect output.
       :returns (parsed nb object, execution errors)
    """
    
    html_path = Path("html/" + path + "-{}-{}".format(SCOPETYPE,PLATFORM) + ".html")
    real_path = Path(path)
    
    with open(real_path, encoding='utf-8') as nbfile:
        nb = nbformat.read(nbfile, as_version=4)
        orig_parameters = extract_parameters(nb)
        params = parameter_values(orig_parameters, SCOPETYPE=SCOPETYPE, PLATFORM=PLATFORM, **kwargs)
        new_nb = replace_definitions(nb, params, execute=False) 

        # allow_errors=False stops execution when an exception is thrown
        # If you're not sure about a tutorial, it's probably a better choice
        ep = ExecutePreprocessor(timeout=None, kernel_name='python3', allow_errors=False)

        ep.preprocess(new_nb, {'metadata': {'path': './'}})

        errors = [[i+1,output] for i,cell in enumerate(new_nb.cells) if "outputs" in cell
                        for output in cell["outputs"]\
                                if output.output_type == "error"]
        
        with open(html_path, "w", encoding='utf-8') as html_file:
            html_exporter = HTMLExporter()
            
            body, res = html_exporter.from_notebook_node(new_nb)
            
            body = strip_ansi(body)
            
            html_file.write(body)
            
        return nb, errors


def _print_tracebacks(errors):
    if errors == []:
        print("Passed all tests!")
    for error in errors:
        print("Test failed in cell {}: {}: {}".format(error[0], error[1]['ename'], error[1]['evalue']))
        for line in error[1]['traceback']:
            print(line)
            
def _get_outputs(nb):
    return [[i,cell] for i,cell in enumerate(nb.cells) if "outputs" in cell]
    
def _print_stderr(nb):
    outputs = _get_outputs(nb)
    printed_output = [[cell[0], output] for cell in outputs for output in cell[1]['outputs'] if ('name' in output and output['name'] == 'stderr')]
    for out in printed_output:
        print("[{}]:\n{}".format(out[0], out[1]['text']))
        
def _print_stdout(nb):
    outputs = _get_outputs(nb)
    printed_output = [[cell[0], output] for cell in outputs for output in cell[1]['outputs'] if ('name' in output and output['name'] == 'stdout')]
    for out in printed_output:
        print("[{}]:\n{}".format(out[0], out[1]['text']))
        
def test_notebook(path,print_stdout=False, print_stderr=False,**kwargs):
    print("Testing: {}:...".format(path), end="")
    nb, errors = _notebook_run(path, **kwargs)
    if errors == []:
        print("PASSED")
    else:
        print("FAILED:")
        _print_tracebacks(errors)
    if print_stdout:
        _print_stdout(nb)
    if print_stderr:
        _print_stderr(nb)

In [4]:
import os
import subprocess
import tempfile
from pathlib import Path
import io
from os import listdir
from os.path import isfile, join
from nbparameterise import extract_parameters, parameter_values, replace_definitions

import nbformat
from nbconvert.preprocessors import ClearOutputPreprocessor
from nbconvert.exporters import NotebookExporter
def clear_notebook(path):
    real_path = Path(path)
    body = ""
    with open(real_path, "r", encoding="utf-8") as nbfile:
        nb = nbformat.read(nbfile, as_version=4)
        orig_parameters = extract_parameters(nb)
        params = parameter_values(orig_parameters, SCOPETYPE="OPENADC", PLATFORM="CWLITEARM")
        new_nb = replace_definitions(nb, params, execute=False)
        co = ClearOutputPreprocessor()
        
        exporter = NotebookExporter()
        node, resources = co.preprocess(new_nb, {'metadata': {'path': './'}})
        body, resources = exporter.from_notebook_node(node, resources)
    with open(real_path, "w", encoding="utf-8") as nbfile:
        nbfile.write(body)
        
def clear_outputs_in_dir(dirpath):
    filter_list = ["Test_Notebook.ipynb", "PA_TVLA_1-Performing_TVLA_Testing_for_Crypto_Validation.ipynb", "PA_Profiling_1_Template_Attacks_HW_Assumption.ipynb" ,"PA_Intro_3-Measuring_SNR_of_Target.ipynb", "PA_HW_CW305.ipynb", "PA_CPA_4-Hardware_Crypto_Attack.ipynb", "Helpful_Code_Blocks.ipynb", "!!Suggested_Completion_Order!!.ipynb", "Fault_4-AES_Differential_Fault_Analysis_Attacks.ipynb"]
    notebook_files = [f for f in listdir("./") if (isfile(join("./", f)) and f.endswith(".ipynb") and f not in filter_list)]
    for file in notebook_files:
        print("Clearing {}".format(file))
        clear_notebook(file)

In [4]:
%%bash
mkdir -p html
cd ../hardware/victims/firmware
rm -rf simpleserial-base-lab*
rm -rf glitch-simple-lab*
rm -rf simpleserial-aes-lab*

In [3]:
testscope = 'OPENADC'
testplat = 'CWLITEARM'
crypt = 'TINYAES128C'

In [21]:
testscope = 'OPENADC'
testplat = 'CWLITEXMEGA'
crypt = 'AVRCRYPTOLIB'

In [1]:
testscope = 'CWNANO'
testplat = 'CWNANO'
crypt = 'TINYAES128C'

In [5]:
test_notebook('PA_Intro_1-Firmware_Build_Setup.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: PA_Intro_1-Firmware_Build_Setup.ipynb:...

CellExecutionError: An error occurred while executing the following cell:
------------------
print(scope.gain)
------------------

[1;31m---------------------------------------------------------------------------[0m
[1;31mAttributeError[0m                            Traceback (most recent call last)
[1;32m<ipython-input-13-46dd3799f84a>[0m in [0;36m<module>[1;34m[0m
[1;32m----> 1[1;33m [0mprint[0m[1;33m([0m[0mscope[0m[1;33m.[0m[0mgain[0m[1;33m)[0m[1;33m[0m[1;33m[0m[0m
[0m
[1;31mAttributeError[0m: 'CWNano' object has no attribute 'gain'
AttributeError: 'CWNano' object has no attribute 'gain'


In [7]:
test_notebook('PA_Intro_2-Instruction_Differences.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: PA_Intro_2-Instruction_Differences.ipynb:...PASSED


In [7]:
test_notebook('PA_SPA_1-Timing_Analysis_with_Power_for_Password_Bypass.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: PA_SPA_1-Timing_Analysis_with_Power_for_Password_Bypass.ipynb:...

CellExecutionError: An error occurred while executing the following cell:
------------------
ret = ""
reset_target(scope)

num_char = target.ser.inWaiting()
while num_char > 0:
    ret += target.ser.read(num_char, 10)
    time.sleep(0.05)
    num_char = target.ser.inWaiting()
    
print(ret)
------------------

[1;31m---------------------------------------------------------------------------[0m
[1;31mUSBError[0m                                  Traceback (most recent call last)
[1;32m<ipython-input-7-edfb898a150e>[0m in [0;36m<module>[1;34m[0m
[0;32m      6[0m     [0mret[0m [1;33m+=[0m [0mtarget[0m[1;33m.[0m[0mser[0m[1;33m.[0m[0mread[0m[1;33m([0m[0mnum_char[0m[1;33m,[0m [1;36m10[0m[1;33m)[0m[1;33m[0m[1;33m[0m[0m
[0;32m      7[0m     [0mtime[0m[1;33m.[0m[0msleep[0m[1;33m([0m[1;36m0.05[0m[1;33m)[0m[1;33m[0m[1;33m[0m[0m
[1;32m----> 8[1;33m     [0mnum_char[0m [1;33m=[0m [0mtarget[0m[1;33m.[0m[0mser[0m[1;33m.[0m[0minW

In [9]:
test_notebook('PA_DPA_1-Hamming_Weight_Measurement.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, CRYPTO_TARGET=crypt)

Testing: PA_DPA_1-Hamming_Weight_Measurement.ipynb:...PASSED


In [10]:
test_notebook('PA_DPA_2-Large_HW_Swings.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, CRYPTO_TARGET=crypt)

Testing: PA_DPA_2-Large_HW_Swings.ipynb:...PASSED


In [ ]:
test_notebook('PA_DPA_3-AES_DPA_Attack.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, CRYPTO_TARGET=crypt)

In [11]:
test_notebook('PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, num_traces=100, CRYPTO_TARGET=crypt)

Testing: PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb:...PASSED


In [12]:
test_notebook('PA_CPA_2-Manual_CPA_Attack.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, num_traces=100, CRYPTO_TARGET=crypt)

Testing: PA_CPA_2-Manual_CPA_Attack.ipynb:...PASSED


In [13]:
test_notebook('PA_CPA_3-Resynchronizing_Data_Traces.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, num_traces=250, CRYPTO_TARGET=crypt)

Testing: PA_CPA_3-Resynchronizing_Data_Traces.ipynb:...PASSED


In [14]:
assert testplat != "CWLITEXMEGA" and testplat != "CW303", "Platform {} is not 32bit!".format(testplat)
test_notebook('PA_CPA_5-32bit_AES.ipynb', SCOPETYPE=testscope, PLATFORM=testplat) #MAY FAIL, takes a long time so may want to skip

Testing: PA_CPA_5-32bit_AES.ipynb:...PASSED


In [16]:
assert testscope != "CWNANO", "Tutorial not yet available for CWNANO"
test_notebook('PA_Multi_1-Breaking_AES-256_Bootloader.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: PA_Multi_1-Breaking_AES-256_Bootloader.ipynb:...PASSED


In [5]:
assert testscope != "CWNANO", "Clock glitching not available on CWNANO"
test_notebook('Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb', SCOPETYPE=testscope, PLATFORM=testplat, sample_size = 5)

Testing: Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb:...PASSED


In [6]:
assert testplat != "CWLITEXMEGA" and testplat != "CW303", "Voltage glitching works poorly on platform {}".format(testplat)
assert testscope != "CWNANO", "Tutorial not yet available for CWNANO"
test_notebook('Fault_2-Introduction_to_Vcc_Glitch_Attacks.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: Fault_2-Introduction_to_Vcc_Glitch_Attacks.ipynb:...PASSED


In [7]:
assert testscope != "CWNANO", "Tutorial not yet available for CWNANO"
test_notebook('Fault_3-Glitch_Buffer_Attacks.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: Fault_3-Glitch_Buffer_Attacks.ipynb:...PASSED


In [9]:
assert testplat != "CWLITEXMEGA" and testplat != "CW303" and testplat != "CWNANO", "RSA fault attack not available on platform {}".format(testplat)
test_notebook('Fault_5-RSA_Fault_Attack.ipynb', SCOPETYPE=testscope, PLATFORM=testplat)

Testing: Fault_5-RSA_Fault_Attack.ipynb:...PASSED


In [5]:
clear_outputs_in_dir("./")

Clearing Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb
Clearing Fault_2-Introduction_to_Vcc_Glitch_Attacks.ipynb
Clearing Fault_3-Glitch_Buffer_Attacks.ipynb
Clearing Fault_5-RSA_Fault_Attack.ipynb
Clearing PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb
Clearing PA_CPA_2-Manual_CPA_Attack.ipynb
Clearing PA_CPA_3-Resynchronizing_Data_Traces.ipynb
Clearing PA_CPA_5-32bit_AES.ipynb
Clearing PA_DPA_1-Hamming_Weight_Measurement.ipynb
Clearing PA_DPA_2-Large_HW_Swings.ipynb
Clearing PA_Intro_1-Firmware_Build_Setup.ipynb
Clearing PA_Intro_2-Instruction_Differences.ipynb
Clearing PA_Multi_1-Breaking_AES-256_Bootloader.ipynb
Clearing PA_SPA_1-Timing_Analysis_with_Power_for_Password_Bypass.ipynb
